In [28]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [17]:
model = 'gpt-4o-mini'
db_name = 'vector_db'

In [18]:
load_dotenv()
OPENAI_KEY = os.getenv('OPENAI_API_KEY')

In [19]:
folders = glob.glob('knowledge-base/*')
text_loader_kwargs = {'encoding': 'utf-8'}

documents = []

for folder in folders :
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata['doc_type'] = doc_type
        documents.append(doc)

In [20]:
text_splitter = CharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 1088, which is longer than the specified 1000


In [21]:
len(chunks)

123

In [22]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: products, company, employees, contracts


In [23]:
embeddings = OpenAIEmbeddings()

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f'Vector Store created {vectorstore._collection.count()} documents')

Vector Store created 123 documents


In [24]:
collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

The vectors have 1,536 dimensions


In [36]:
llm = ChatOpenAI(temperature=0.7, model_name=model)
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
retriever = vectorstore.as_retriever()
converstaion_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [35]:
query = 'Can you descripe Insurelm in few sentences'
result = converstaion_chain.invoke({'question':query})
print(result['answer'])

Insurellm is an innovative insurance tech firm founded by Avery Lancaster in 2015, specializing in disruption within the insurance industry through its software products. The company offers four main products: Carllm for auto insurance, Homellm for home insurance, Rellm for the reinsurance sector, and Marketllm, a marketplace connecting consumers with insurance providers. With a workforce of 200 employees and over 300 clients worldwide, Insurellm aims to provide cutting-edge solutions and exceptional customer support in the insurance sector.


In [34]:
def chat(message,history):
    result = converstaion_chain.invoke({'question':message})
    return result['answer']

In [ ]:
view = gr.ChatInterface(fn=chat).launch()